# **Travelling Salesman Problem (TSP):**

**Problem Statement:**

Given a set of cities and the distance between every pair of cities, the problem is to find the shortest possible route that visits every city exactly once and returns to the starting point.


**General Modeling:**

$\\ $

$Objective\ Function:\\ $

$Min: Z = \sum_{i=1}^{n} \sum_{j=1, j \neq i}^{n} c_{ij} \cdot x_{ij}$

$\\ $

$Constraints:\\ $


$\sum_{j=1, j \neq i}^{n} x_{ij} = 1$ for all $i = 1$ to $n$

$\sum_{i=1, i \neq j}^{n} x_{ij} = 1$ for all $j = 1$ to $n$

$x_{ij} = 0$ or $1$

Suppose we have 4 cities: A, B, C, and D, with the following distances (in arbitrary units):

•	Distance from A to B: 10

•	Distance from A to C: 15

•	Distance from A to D: 20

•	Distance from B to C: 35

•	Distance from B to D: 25

•	Distance from C to D: 30

We want to find the shortest tour that starts at one of the cities, visits each city exactly once, and returns to the starting city.

In [1]:
!pip install pulp
from pulp import *
import numpy as np
import pandas as pd

In [2]:
#Defining the Problem

#List of cities
cities = ['A', 'B', 'C', 'D']

#Distance matrix (dictionary) with distances between cities
distances = {
    ('A', 'B'): 10,
    ('B', 'A'): 10,  # Add the reverse direction
    ('A', 'C'): 15,
    ('C', 'A'): 15,  # Add the reverse direction
    ('A', 'D'): 20,
    ('D', 'A'): 20,  # Add the reverse direction
    ('B', 'C'): 35,
    ('C', 'B'): 35,  # Add the reverse direction
    ('B', 'D'): 25,
    ('D', 'B'): 25,  # Add the reverse direction
    ('C', 'D'): 30,
    ('D', 'C'): 30,  # Add the reverse direction
}

In [3]:
#Defining Variables

x = {}
for city1 in cities:
    for city2 in cities:
        if city1 != city2:
            x[(city1, city2)] = LpVariable(f"x_{city1}_{city2}",  cat='Binary')

In [4]:
#Defining Objective Function

tsp = LpProblem('TSP', LpMinimize)
tsp += lpSum(distances[(city1, city2)] * x[(city1, city2)] + distances[(city2, city1)] * x[(city2, city1)]
             for city1 in cities for city2 in cities if city1 != city2)

In [5]:
#Constraints

for city in cities:
    tsp += lpSum(x[(city1, city)] for city1 in cities if city1 != city) == 1
    tsp += lpSum(x[(city, city2)] for city2 in cities if city2 != city) == 1

In [6]:
#Model summary

tsp

TSP:
MINIMIZE
20*x_A_B + 30*x_A_C + 40*x_A_D + 20*x_B_A + 70*x_B_C + 50*x_B_D + 30*x_C_A + 70*x_C_B + 60*x_C_D + 40*x_D_A + 50*x_D_B + 60*x_D_C + 0
SUBJECT TO
_C1: x_B_A + x_C_A + x_D_A = 1

_C2: x_A_B + x_A_C + x_A_D = 1

_C3: x_A_B + x_C_B + x_D_B = 1

_C4: x_B_A + x_B_C + x_B_D = 1

_C5: x_A_C + x_B_C + x_D_C = 1

_C6: x_C_A + x_C_B + x_C_D = 1

_C7: x_A_D + x_B_D + x_C_D = 1

_C8: x_D_A + x_D_B + x_D_C = 1

VARIABLES
0 <= x_A_B <= 1 Integer
0 <= x_A_C <= 1 Integer
0 <= x_A_D <= 1 Integer
0 <= x_B_A <= 1 Integer
0 <= x_B_C <= 1 Integer
0 <= x_B_D <= 1 Integer
0 <= x_C_A <= 1 Integer
0 <= x_C_B <= 1 Integer
0 <= x_C_D <= 1 Integer
0 <= x_D_A <= 1 Integer
0 <= x_D_B <= 1 Integer
0 <= x_D_C <= 1 Integer

In [11]:
#Solving model

tsp.solve()
print('Optimal Solution:', pulp.value(tsp.objective))

Optimal Solution: 160.0


In [8]:
#Optimal values of variables

for variables in tsp.variables():
  print('{}:{}'.format(variables.name, variables.varValue))

x_A_B:0.0
x_A_C:1.0
x_A_D:0.0
x_B_A:1.0
x_B_C:0.0
x_B_D:0.0
x_C_A:0.0
x_C_B:0.0
x_C_D:1.0
x_D_A:0.0
x_D_B:1.0
x_D_C:0.0


In [10]:
VNames = []
for variables in tsp.variables():
  VNames.append(variables.name)

VValue = []
for variables in tsp.variables():
  VValue.append(int(variables.varValue))

data = {'Variables': VNames, 'Value': VValue}
pd.DataFrame(data, index = range(1, len(VValue)+1))

,Variables,Value
1,x_A_B,0
2,x_A_C,1
3,x_A_D,0
4,x_B_A,1
5,x_B_C,0
6,x_B_D,0
7,x_C_A,0
8,x_C_B,0
9,x_C_D,1
10,x_D_A,0


In [12]:
print('Current Status: ', LpStatus[tsp.status])

Current Status:  Optimal
